In [1]:
import os
import numpy as np
from PIL import Image
from typing import Any, Tuple, Optional, Callable
import tensorflow as tf
import cv2
import pandas as pd
from torch.utils.data import Dataset
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
# Constant list
PATH_IMG = './images'
SAMPLE_FRAC = 0.11
RANDSEED = 33

In [3]:
def read_csv(path: str) -> pd.DataFrame:
    '''
    Read a csv file.

    Args:
        path (str): Path to the csv file.

    Returns:
        pd.DataFrame: Dataframe with the csv file data.
    '''

    assert os.path.exists(path), f'CSV file not found: {path}!'
    assert os.path.splitext(path)[-1] == '.csv', f'Unsupported file type {os.path.splitext(path)[-1]}!'
    return pd.read_csv(path)

class ImageDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, images_folder: str = PATH_IMG, transform: Optional[Callable] = None, target_transform: Optional[Callable] = None) -> None:
        '''
        Image dataset.

        Args:
            dataframe (pd.DataFrame): Dataframe with the image filenames and labels.
            images_folder (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
            target_transform (callable, optional): Optional transform to be applied on a target.
        '''
        assert 'Filename' in dataframe.columns, f'Filename column not found!'
        assert os.path.exists(images_folder), f'Image folder not found: {images_folder}!'

        self.dataframe = dataframe
        self.images_folder = images_folder
        self.transform = transform
        self.target_transform = target_transform

        data = []
        targets = []

        for i, sample in dataframe.iterrows():
            image = cv2.imread(os.path.join(images_folder, sample['Filename']))
            data.append(image)

            targets.append(int(sample['Label']) if 'Label' in sample else -1)

        self.data = data
        self.targets = targets

    def __len__(self) -> int:
        '''
        Returns:
            int: Length of the dataset.
        '''
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        '''
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is class_index of the target class. For the public test set, target is a class from [0, 1, 2, 3, 4, 5, 6, 7, 8]. For the private test set (before releasing the test set labels), target is -1.
        '''
        img = self.data[index]
        target = self.targets[index]

        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

In [4]:
public_dataframe = read_csv('assignment_7_public.csv')
df_0 = public_dataframe[public_dataframe["Label"]==0]
df_1 = public_dataframe[public_dataframe["Label"]==1]
df_2 = public_dataframe[public_dataframe["Label"]==2]
df_3 = public_dataframe[public_dataframe["Label"]==3]
df_4 = public_dataframe[public_dataframe["Label"]==4]
df_5 = public_dataframe[public_dataframe["Label"]==5]
df_6 = public_dataframe[public_dataframe["Label"]==6]
df_7 = public_dataframe[public_dataframe["Label"]==7]
df_8 = public_dataframe[public_dataframe["Label"]==8]



df_0_sam = df_0.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_1_sam = df_1.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_2_sam = df_2.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_3_sam = df_3.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_4_sam = df_4.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_5_sam = df_5.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_6_sam = df_6.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_7_sam = df_7.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)
df_8_sam = df_8.sample(frac=SAMPLE_FRAC, random_state=RANDSEED)


# Concatenate the dataframes along the rows
concatenated_df = pd.concat([df_0_sam, df_1_sam, df_3_sam, df_4_sam, df_5_sam,
                             df_6_sam, df_7_sam, df_8_sam], axis=0)

concatenated_df.value_counts(['Label']).sort_index(ascending=True).to_frame()



,count
Label,
0,1032
1,996
3,1079
4,873
5,1249
6,833
7,958
8,1367


In [5]:
# # Set the GPU device
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')

# tf.config.experimental.set_memory_growth(gpu_devices[0], True)
# print("Num GPUs:", len(gpu_devices))

# print("Support cuda:", tf.test.is_built_with_cuda())

In [6]:
public_dataset = ImageDataset(concatenated_df)

print('Image', type(public_dataset[0][0]), public_dataset[0][0].size) # Image <class 'PIL.Image.Image'> (28, 28)
print('Target', type(public_dataset[0][1])) # Target <class 'int'>
print('Length', len(public_dataset)) # Length 85744

Image <class 'PIL.Image.Image'> (28, 28)
Target <class 'int'>
Length 8387


In [7]:
# Assuming the label column is named 'label' in public_dataframe
labels = np.array(concatenated_df['Label'])

# Extract the image data from the ImageDataset
images = np.asarray([np.array(sample[0]) for sample in public_dataset])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)

# Normalize the input data (assuming it is in the range of 0-255)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

input_shape = (28, 28, 3)

# Define the model architecture
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(9, activation="softmax"),
    ]
)

model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


batch_size = 64
epochs = 100
# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Evaluate the model on the test data
_, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)


Epoch 1/100
118/118 [==============================] - 3s 18ms/step - loss: 1.9366 - accuracy: 0.2442 - val_loss: 1.6706 - val_accuracy: 0.4184
Epoch 2/100
118/118 [==============================] - 2s 18ms/step - loss: 1.5210 - accuracy: 0.4140 - val_loss: 1.3080 - val_accuracy: 0.5089
Epoch 3/100
118/118 [==============================] - 2s 18ms/step - loss: 1.3438 - accuracy: 0.4903 - val_loss: 1.1828 - val_accuracy: 0.5614
Epoch 4/100
118/118 [==============================] - 2s 18ms/step - loss: 1.2373 - accuracy: 0.5326 - val_loss: 1.0918 - val_accuracy: 0.5721
Epoch 5/100
118/118 [==============================] - 2s 19ms/step - loss: 1.1509 - accuracy: 0.5672 - val_loss: 1.0107 - val_accuracy: 0.6162
Epoch 6/100
118/118 [==============================] - 2s 19ms/step - loss: 1.0735 - accuracy: 0.5992 - val_loss: 0.9668 - val_accuracy: 0.6174
Epoch 7/100
118/118 [==============================] - 2s 19ms/step - loss: 1.0310 - accuracy: 0.6170 - val_loss: 0.9480 - val_accuracy:

In [9]:
private_dataframe = read_csv('assignment_7_private.csv')
private_dataset = ImageDataset(private_dataframe)

print('Image', type(private_dataset[0][0])) # Image <class 'PIL.Image.Image'> (28, 28)
print('Length', len(private_dataset)) # Length 21436



# Convert images to numpy arrays and normalize
predict_imgages = np.asarray([np.array(sample[0]) for sample in private_dataset])
predict_imgages = predict_imgages.astype('float32') / 255.0

# Make predictions using the trained model
predicted_labels = model.predict(predict_imgages)

# Get the predicted labels
preds = np.argmax(predicted_labels, axis=1)


submission = pd.DataFrame({'Label': preds})
submission.to_csv('assignment_7.csv', index=True, index_label='Id')

Image <class 'PIL.Image.Image'>
Length 21436
670/670 [==============================] - 2s 4ms/step
